In [1]:
#coding:utf-8
%reload_ext autoreload 
%autoreload 2
import os
import sys

project_basedir = '..'
sys.path.append(project_basedir)
from cchess_zero import cbf
from cchess_zero import board_visualizer
from matplotlib import pyplot as plt
import os
from cchess_zero.gameboard import *
from net import resnet
import common
from gameplays import gameplay,players
from gameplays.analysis import get_km_json
import json
%matplotlib inline

/usr/local/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [234]:
! ls '../data/validate/2018-07-30_16-13-25/' | head

2018-07-30_17-21-26_340_mcts-mcts_oldnet-newnet_b.cbf
2018-07-30_17-21-39_624_mcts-mcts_newnet-oldnet_b.cbf
2018-07-30_17-24-55_869_mcts-mcts_oldnet-newnet_w.cbf
2018-07-30_17-25-32_104_mcts-mcts_oldnet-newnet_b.cbf
2018-07-30_17-28-32_788_mcts-mcts_oldnet-newnet_w.cbf
2018-07-30_17-29-14_897_mcts-mcts_oldnet-newnet_b.cbf
2018-07-30_17-29-38_131_mcts-mcts_newnet-oldnet_b.cbf
2018-07-30_17-30-47_629_mcts-mcts_oldnet-newnet_peace.cbf
2018-07-30_17-30-49_887_mcts-mcts_newnet-oldnet_peace.cbf
2018-07-30_17-30-57_325_mcts-mcts_oldnet-newnet_b.cbf


In [229]:

fullpath = '../data/validate/2018-07-30_16-13-25/2018-07-30_19-19-34_573_mcts-mcts_newnet-oldnet_w.cbf'
moves = cbf.cbf2move(fullpath)
fname = fullpath.split('/')[-1]
print(fname)
state = gameplay.GameState()

2018-07-30_19-19-34_573_mcts-mcts_newnet-oldnet_w.cbf


In [230]:
from PIL import Image
import imageio
imgs_for_gif = []
state = gameplay.GameState()
statestr = 'RNBAKABNR/9/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5c1/9/rnbakabnr'
for i in moves[:]: 
    statestr = GameBoard.sim_do_action(i,statestr)
    imgs_for_gif.append(board_visualizer.get_board_img(statestr))
    state.do_move(i)
imgs_for_gif = imgs_for_gif[-10:]
imageio.mimsave('../data/gif/{}.gif'.format(fname[:-4]), imgs_for_gif,duration=0.5)

In [231]:
i,len(moves)

('f7f8', 262)

In [232]:
def display_gif(fn):
    from IPython import display
    return display.HTML('<img src="{}">'.format(fn))

In [233]:
display_gif('../data/gif/{}.gif'.format(fname[:-4]))

# ANALYSIS

In [129]:
network = resnet.get_model(os.path.join(project_basedir,
    '../data/prepare_weight/2018-07-25_22-18-46'[3:]),
    common.board.create_uci_labels(),GPU_CORE=[None],FILTERS=128,NUM_RES_LAYERS=7)

0
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
inside res net [None, 10, 9, 128]
INFO:tensorflow:Restoring parameters from ../data/prepare_weight/2018-07-25_22-18-46


In [134]:
network_player = players.NetworkPlayer(state.currentplayer,network,n_playout=400)

In [135]:
network_player.make_move(state=state,actual_move=False)

('e1d1', 0.26717197106045204)

In [136]:
childs = network_player.mcts_policy._root._children
for move in childs:
    val = childs[move]
    print(move,val._n_visits,val._P,val._Q)

b0a2 0 4.9169368e-05 0
b0d1 12 0.018395053 0.24550490205486614
b0c2 0 0.00020596824 0
c3b3 0 1.0081266e-05 0
c3d3 0 8.3958175e-06 0
c3c0 0 2.3718806e-06 0
c3c1 0 1.5922126e-06 0
c3c2 0 3.3794254e-07 0
c3c5 0 9.780939e-07 0
e1d1 266 0.32702333 0.4471466393512848
e1f1 34 0.49376035 -1.0
e1e0 0 0.00011869673 0
e1e2 1 0.004035793 -1.0
e5c5 0 4.2461957e-06 0
e5d5 0 0.00036026465 0
e5f5 0 0.0009615962 0
e5g5 0 6.967882e-06 0
e5h5 0 4.8325285e-07 0
e5e3 0 2.0202579e-05 0
e5e4 0 0.000933298 0
e5e6 0 0.0001555884 0
e5e7 0 0.00019506543 0
g7f7 0 9.827086e-06 0
g7h7 0 0.0037287443 0
g7g8 0 3.452016e-05 0
h2d2 0 0.00032670892 0
h2e2 0 0.00076633267 0
h2f2 0 0.0005389656 0
h2g2 0 0.00011242429 0
h2h0 0 2.3433591e-05 0
h2h1 0 0.00010361684 0
h2h3 0 2.8984734e-05 0
h2h4 0 5.418088e-05 0
h2h5 0 1.1451802e-05 0
h2h6 0 2.3306018e-05 0
h2h7 0 0.0004233554 0
h2h8 0 7.880476e-05 0
h2h9 30 0.0075685116 0.4245055218537649
i2h0 0 1.2329712e-05 0
i2h4 0 0.0008276026 0
i5h5 0 0.00010810893 0
i5i6 0 1.7538223e-0

In [28]:
datajson = get_km_json(network_player.mcts_policy,state.statestr,depth=40)

In [29]:
with open('../data/mstc_vis/sample.km','w') as whdl:
    json.dump(datajson,whdl)